In [1]:
import math

from openfermionpyscf import run_pyscf
from openfermion.transforms import binary_code_transform, bravyi_kitaev_code, get_fermion_operator
from openfermion.hamiltonians import MolecularData
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.utils import count_qubits
from pyscf import gto, scf, mcscf

from helper_functions import *
from XBK_method import *

In [2]:
#create molecule
name = 'H3'
charge = 1
multiplicity = 1
basis = '631g'

bond_length = 1.1
geometry = get_molGeometry(name, bond_length)

molecule = MolecularData(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge
)

In [3]:
#run RHF calculations
molecule = run_pyscf(molecule, run_scf=True)
hf_energy = float(molecule.hf_energy)
hf_data = molecule._pyscf_data['scf']

print(hf_energy)

-1.236374104407568


In [4]:
#define active space
n_active_electrons = 2
n_active_orbitals = 3
occupied_indices, active_indices = get_active_space(molecule, n_active_electrons, n_active_orbitals)

#run CASCI calculations
casci_data = hf_data.CASCI(n_active_orbitals, n_active_electrons).run(verbose=False)
casci_energy = float(casci_data.e_tot)

print(casci_energy)

-1.2553148319654226


In [5]:
#convert to fermionic Hamiltonian
molecular_H = molecule.get_molecular_hamiltonian(occupied_indices=occupied_indices, active_indices=active_indices)
if molecular_H[()] == None:
    molecular_H[()] = 0
fermionic_H = get_fermion_operator(molecular_H)

#add penalty term to ensure correct number of electrons in ground state
weight = 5
penalty_term = FermionOperator('', n_active_electrons)

for i in range(molecular_H.n_qubits):
    penalty_term += FermionOperator(str(i)+'^ '+str(i), -1)
fermionic_H += weight*penalty_term**2

print(fermionic_H)

21.443210575236364 [] +
-21.62379810334505 [0^ 0] +
5.0 [0^ 0 0^ 0] +
5.0 [0^ 0 1^ 1] +
5.0 [0^ 0 2^ 2] +
5.0 [0^ 0 3^ 3] +
5.0 [0^ 0 4^ 4] +
5.0 [0^ 0 5^ 5] +
0.28400576352308204 [0^ 0^ 0 0] +
0.051182507589769055 [0^ 0^ 2 2] +
0.05118250758976903 [0^ 0^ 4 4] +
0.28400576352308204 [0^ 1^ 1 0] +
0.051182507589769055 [0^ 1^ 3 2] +
0.05118250758976903 [0^ 1^ 5 4] +
0.051182507589769055 [0^ 2^ 0 2] +
0.22520695976033195 [0^ 2^ 2 0] +
0.004819967321007128 [0^ 2^ 2 2] +
-0.02502497525177295 [0^ 2^ 2 4] +
-0.025024975251772906 [0^ 2^ 4 2] +
-0.004819967321007212 [0^ 2^ 4 4] +
0.051182507589769055 [0^ 3^ 1 2] +
0.22520695976033195 [0^ 3^ 3 0] +
0.004819967321007128 [0^ 3^ 3 2] +
-0.02502497525177295 [0^ 3^ 3 4] +
-0.025024975251772906 [0^ 3^ 5 2] +
-0.004819967321007212 [0^ 3^ 5 4] +
0.05118250758976903 [0^ 4^ 0 4] +
-0.025024975251772906 [0^ 4^ 2 2] +
-0.004819967321007212 [0^ 4^ 2 4] +
0.225206959760332 [0^ 4^ 4 0] +
-0.004819967321007343 [0^ 4^ 4 2] +
0.025024975251772854 [0^ 4^ 4 4] +
0.0

In [6]:
#convert to Pauli operator Hamiltonian
binary_code = bravyi_kitaev_code(molecular_H.n_qubits)
qubit_H = binary_code_transform(fermionic_H, binary_code)
qubit_H.compress()

#apply symmetry reductions and calculate minimum eigenvalue (should be equal to CASCI energy)
sectors = taper_qubits(qubit_H)
qubit_H, min_eigenvalue = sector_with_ground(sectors)
m = count_qubits(qubit_H)

print(min_eigenvalue, '\n')
print(qubit_H)

-1.2553148319654395 

11.748849714003 [] +
-0.002409983660503559 [X0] +
0.0024099836605036154 [X0 X1 X2] +
-0.012512487625886455 [X0 X1 Z2 X3] +
0.002409983660503559 [X0 X1 Z2 Z3] +
-0.012512487625886455 [X0 X1 X3] +
0.002409983660503559 [X0 X1 Z3] +
0.0024099836605036154 [X0 Y1 Y2] +
0.012512487625886429 [X0 Z1 X2 X3] +
-0.002409983660503615 [X0 Z1 X2 Z3] +
0.002409983660503669 [X0 Z1 Z2] +
-0.012512487625886429 [X0 Z1 X3] +
0.002409983660503669 [X0 Z1 Z3] +
0.012512487625886429 [X0 X2 X3] +
-0.002409983660503615 [X0 X2 Z3] +
0.012512487625886477 [X0 Z2 X3] +
-0.002409983660503559 [X0 Z2 Z3] +
-0.012512487625886453 [Y0 X1 X2 Y3] +
-0.012512487625886453 [Y0 X1 Y2 X3] +
0.0024099836605036154 [Y0 X1 Y2 Z3] +
0.012512487625886453 [Y0 Y1 X2 X3] +
-0.0024099836605036154 [Y0 Y1 X2 Z3] +
-0.012512487625886453 [Y0 Y1 Y2 Y3] +
-0.012512487625886455 [Y0 Y1 Z2 X3] +
0.002409983660503669 [Y0 Y1 Z2 Z3] +
-0.012512487625886455 [Y0 Y1 X3] +
0.002409983660503669 [Y0 Y1 Z3] +
0.012512487625886477 [Y0 Z

In [7]:
from openfermion import get_sparse_operator

result = XBK_transform(qubit_H, r=1, p=1)

In [8]:
result

11.748849714003 [] +
2.642002881761541 [Z0] +
-2.612603479880166 [Z0 Z1] +
7.3087640214886465 [Z0 Z1 Z2] +
2.5891565890198076 [Z0 Z1 Z2 Z3] +
-7.5474696044780725 [Z0 Z1 Z3] +
-2.612603479880166 [Z0 Z2] +
-2.612603479880166 [Z0 Z2 Z3] +
2.6086122081219623 [Z0 Z3] +
-7.5474696044780725 [Z1] +
7.3087640214886465 [Z1 Z2] +
2.5891565890198076 [Z1 Z2 Z3] +
-2.612603479880166 [Z1 Z3] +
-7.5474696044780725 [Z2] +
-7.5474696044780725 [Z2 Z3] +
2.6086122081219623 [Z3]

In [ ]:
import warnings
warnings.filterwarnings("ignore")
### XBK method ###

#set r value
r = 1

#construct qubit Hamiltonians and C terms for XBK method
qubit_Hs, qubit_Cs = [],[]
for p in range(int(math.ceil(r/2+1))):
    qubit_Hs += [XBK_transform(qubit_H, r, p)]
    qubit_Cs += [construct_C(m, r, p)]

#run XBK method
XBK_energy, ground_state = XBK(qubit_Hs, qubit_Cs, r, starting_lam=0, num_samples=1000, strength=1e3, verbose=False)

print(XBK_energy)
print(ground_state) #ground state in rm-qubit space

AnalogChain(t=693.492130864141, support=(<QubitSupportType.GLOBAL: 'global'>,))
├── AnalogChain(t=160.484979153848, support=(<QubitSupportType.GLOBAL: 'global'>,))
│   ├── ConstantAnalogRotation(α=t0, t=318.309886183791*t0, support=(<QubitSupportType.GLOBAL: 'global'>,), 
│   │   Ω=3.14159265358979, δ=0, φ=0)
│   └── ConstantAnalogRotation(α=s0, t=318.309886183791*s0, support=(<QubitSupportType.GLOBAL: 'global'>,), Ω=0, 
│       δ=3.14159265358979, φ=0.0)
└── AnalogChain(t=533.007151710293, support=(<QubitSupportType.GLOBAL: 'global'>,))
    ├── ConstantAnalogRotation(α=t1, t=318.309886183791*t1, support=(<QubitSupportType.GLOBAL: 'global'>,), 
    │   Ω=3.14159265358979, δ=0, φ=0)
    └── ConstantAnalogRotation(α=s1, t=318.309886183791*s1, support=(<QubitSupportType.GLOBAL: 'global'>,), Ω=0, 
        δ=3.14159265358979, φ=0.0)


2025-03-09 11:18:16 INFO     Initializing Accelerator                                           ]8;id=425584;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/train_utils/distribution.py\distribution.py]8;;\:]8;id=940945;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/train_utils/distribution.py#108\108]8;;\

                    INFO     =============================                                      ]8;id=178654;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/train_utils/distribution.py\distribution.py]8;;\:]8;id=166494;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/train_utils/distribution.py#109\109]8;;\

                    INFO      Node, Device                : Unknown, cpu                        ]8;id=926597;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/train_utils/distribution.py\distribution.py]8;;\:]8;id=346904;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/train_utils/distribution.py#110\110]8;;\

                    INFO      Rank, Local Rank, World Size: 0, None, 1                          ]8;id=781717;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/train_utils/distribution.py\distribution.py]8;;\:]8;id=644902;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/train_utils/distribution.py#115\115]8;;\

...

/Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


2025-03-09 11:18:41 INFO     Writing QuantumModel checkpoint                                        ]8;id=619776;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/callbacks/saveload.py\saveload.py]8;;\:]8;id=311028;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/callbacks/saveload.py#106\106]8;;\
                             model_QuantumModel_ckpt_100_device_cpu.pt                                             

                    INFO     Writing NGOpt to checkpoint opt_NGOpt_ckpt_100_device_cpu.pt           ]8;id=803142;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/callbacks/saveload.py\saveload.py]8;;\:]8;id=893662;file:///Users/arshkumar/miniforge3/envs/pasqal/lib/python3.12/site-packages/qadence/ml_tools/callbacks/saveload.py#116\116]8;;\

{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 0, 's2': 1, 's2*s1': 0, 's3*s0': 0, 's1': 0}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 0, 's2': 1, 's2*s1': 0, 's3*s0': 0, 's1': 1}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 0, 's2': 1, 's2*s1': 0, 's3*s0': 1, 's1': 0}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 0, 's2': 1, 's2*s1': 1, 's3*s0': 0, 's1': 0}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 1, 's2': 0, 's2*s1': 0, 's3*s0': 0, 's1': 0}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 1, 's2': 0, 's2*s1': 0, 's3*s0': 0, 's1': 1}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 1, 's2': 0, 's2*s1': 0, 's3*s0': 1, 's1': 0}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 1, 's2': 0, 's2*s1': 1, 's3*s0': 0, 's1': 0}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 1, 's2': 1, 's2*s1': 0, 's3*s0': 0, 's1': 0}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 1, 's2': 1, 's2*s1': 0, 's3*s0': 0, 's1': 1}
{'auxs2,s1': 0, 'auxs3,s0': 0, 's3': 0, 's0': 1, 's2': 1, 's2*s1': 0, 's3*s0': 1, 's1': 0}

/Users/arshkumar/MyPython/Pasqal_Hackathon_Feb25_Team_04-Henry/XBK_method.py:139: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  index = int(solutions[['energy']].idxmin())


In [ ]:
new_map = {}
for i in range(8):
    new_map['s'+str(i)] = mapping['s'+str(i)]
new_map

In [ ]:
solutions

In [ ]:
solutions.iloc[0]['sample']

In [ ]:
constant

In [ ]:
for i in range(8):
    print(solutions.iloc[0]['sample']['s'+str(i)])

In [ ]:
x = np.array([1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
XBK_energy

In [ ]:
x.T@Q@x

In [ ]:
new_map = {'s1': 0,'s2': 3,'s0': 5,'s3': 7}

In [ ]:
mapping

In [ ]:
from openfermion import get_sparse_operator